# Downloading Recipe Database from EDAMAM

Using our credentials to get recipe base

In [ ]:
from py_edamam import PyEdamam
import pandas as pd

e = PyEdamam(recipes_appid='3111de14',
           recipes_appkey='89bad7648b3b6ca6c18312f41bc1cfcf',
            food_appid="07d50733",
                 food_appkey="80fcb49b500737827a9a23f7049653b9")

Defining the list of queries to go through and export (note there is a limit on the export hence I break the list in 11 sets)

In [75]:
list_queries = ["pasta","pizza","lasagne","soup","salad","beef","ground beef","chicken","chicken breast","pork","pork chop","lamb","fish","schrimps","rice","quinoa","brocoli","tomatoes","pies","vegetarian","sweet potato","avocado","italian","mexican","tacos","quesadilla","british","japanese","vegan","dairy free","gluten free","low carb","high protein","bake","roast","grilled"]

In [12]:
list_query_1 = ["pasta","pizza","lasagne","soup","salad"]
list_query_2 = ["beef","ground beef","chicken","chicken breast","pork"]
list_query_3 = ["pork chop","lamb","fish","schrimps","rice"]
list_query_4 = ["quinoa","brocoli","tomatoes","pies","vegetarian"]
list_query_5 = ["sweet potato","avocado","italian","mexican","tacos"]
list_query_6 = ["quesadilla","british","japanese","vegan"]
list_query_7 = ["dairy free","gluten free","low carb","high protein"]
list_query_8 = ["bake","roast","grilled"]
list_query_9 = ["risotto","burger","cajun","shepperd pie", "carrot"]
list_query_10 = ["sushi","salmon","seabass","trout"]
list_query_11 = ["tofu","chickpea","lentil","dahl"]

In [52]:
list_queries = [list_query_1,list_query_2,list_query_3,list_query_4,list_query_5,list_query_6,list_query_7,list_query_8]

Exporting list of recipes and ingredient names as well as another file with weights included. Additionnally we print the nutrional values for each recipes.

In [13]:
df_ingredient = pd.DataFrame()
df_ingredient_weight = pd.DataFrame()
df_nutrient=pd.DataFrame(columns=["nutrient_list","Nutrient_notclean","tmp","Value","Unit","Nutrient","recipe"])
for query in list_query_11:
    print(query)
    for recipe in e.search_recipe(query):   #go through the list of all recipes
        #getting all the ingredient names
        dfitmp = pd.DataFrame(data = recipe.ingredient_names)
        dfitmp=dfitmp.T
        dfitmp.insert(0,"recipe",str(recipe))
        dfitmp.insert(0,"query",str(query))
        df_ingredient=df_ingredient.append(dfitmp)
        #getting all the ingredient names and weights
        dfiqtmp=pd.DataFrame(data = recipe.ingredient_quantities)
        dfiqtmp = dfiqtmp.iloc[:, 0:2]
        dfiqtmp=dfiqtmp.T
        dfiqtmp.insert(0,"recipe",str(recipe))
        dfiqtmp.insert(0,"query",str(query))
        df_ingredient_weight=df_ingredient_weight.append(dfiqtmp)
        #getting all nutrients
        Nutrients=recipe.totalNutrients
        NutrientsList =[]
        recipe_name = recipe
        for n in Nutrients:                                             #for each recipe go through all nutrients
            NutrientsList+=[str(n)]                                     #transforming Nutrient type into a string
        dfntmp = pd.DataFrame(data = NutrientsList, columns=["nutrient_list"])      #creating df from the list of nutrients
        dfntmp[["Nutrient_notclean","tmp"]]=dfntmp.nutrient_list.str.split("*",expand=True)      #splitting name and measurement using '*' also to avoid issues with Vitamins
        dfntmp[["Value","Unit"]]=dfntmp.tmp.str.split(expand=True)                   #splitting measurement into value and unit
        Nutrients2 =[]                                                       #creating a new list to remove space at the end of the nutrients
        for n in dfntmp.Nutrient_notclean:
            Nutrients2+=[n.strip()]
        dfntmp["Nutrient"]=Nutrients2                                            #adding this column to df
        dfntmp["recipe"]=str(recipe_name)                                            #adding column with recipe name
        dfntmp["Value"]=dfntmp["Value"].astype(float)                                #converting Value for string to flaot to be able to do pivot
        df_nutrient=df_nutrient.append(dfntmp)
    
df_pivot_nutrient=df_nutrient.pivot_table('Value', 'recipe', 'Nutrient')

tofu
chickpea
lentil
dahl


In [14]:
df_ingredient.to_csv(path_or_buf='C:/Users/flori/Documents/Master/LSE/Dissertation/Python/recipe_ingredients_list_query_11.csv', index=True)
df_ingredient_weight.to_csv(path_or_buf='C:/Users/flori/Documents/Master/LSE/Dissertation/Python/recipe_ingredients_weights_list_query_11.csv', index=True)
df_pivot_nutrient.to_csv(path_or_buf='C:/Users/flori/Documents/Master/LSE/Dissertation/Python/recipe_nutrients_list_query_11.csv', index=True)

Combining all csv files into one

In [88]:
import os
import glob
os.chdir("C:/Users/flori/Documents/Master/LSE/Dissertation/Python/Recipes/")

In [89]:
extension = 'csv'
all_filenames_ingredients = [i for i in glob.glob('recipe_ingredients_list_*.{}'.format(extension))]
all_filenames_ingredients_weights = [i for i in glob.glob('recipe_ingredients_weights_*.{}'.format(extension))]
all_filenames_nutrients = [i for i in glob.glob('recipe_nutrients_*.{}'.format(extension))]

In [96]:
combined_csv_ingredients = pd.concat([pd.read_csv(f) for f in all_filenames_ingredients ],sort=False)
combined_csv_ingredients_weights = pd.concat([pd.read_csv(f) for f in all_filenames_ingredients_weights ],sort=False)
combined_csv_nutrients = pd.concat([pd.read_csv(f) for f in all_filenames_nutrients ],sort=False)
#export to csv
combined_csv_ingredients.to_csv( "recipe_ingredients.csv", index=False, encoding='utf-8-sig')
combined_csv_ingredients_weights.to_csv( "recipe_ingredients_weights.csv", index=False, encoding='utf-8-sig')
combined_csv_nutrients.to_csv( "recipe_nutrients.csv", index=False, encoding='utf-8-sig')

I need the servings for all recipes to adjust for individuals (run belowcommand for all 11 data sets)

In [16]:
df_urls = pd.DataFrame()
for query in list_query_11:
    print(query)
    for recipe in e.search_recipe(query): 
        dfutmp = pd.DataFrame(data = [query,recipe,recipe.url])
        dfutmp = dfutmp.T
        df_urls = df_urls.append(dfutmp)
        
df_urls.to_csv(path_or_buf='C:/Users/flori/Documents/Master/LSE/Dissertation/Python/recipe_urls_list_query_11.csv', index=True)

tofu
chickpea
lentil
dahl


In [120]:
extension = 'csv'
all_filenames_urls = [i for i in glob.glob('recipe_urls_*.{}'.format(extension))]
combined_csv_urls = pd.concat([pd.read_csv(f) for f in all_filenames_urls ],sort=False)
combined_csv_urls.to_csv( "recipe_urls.csv", index=False, encoding='utf-8-sig')